### DrivenData Pump it Up Challenge:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
#Reading the data files and assigning them to variables - we have 3 csv files
#test_values_data = pd.read_csv('Data/test set values.csv')
# After talking to Blake, we decided not to use the test data
training_label = pd.read_csv('Data/training set labels.csv')
training_values_data = pd.read_csv('Data/training set values.csv')

In [3]:
# Merge training_values_data and training_label dfs to have one df
train_data = pd.merge(training_values_data, training_label, on = "id")
#train_data = train_data.fillna(train_data.median())

In [4]:
# Deleting training_label df
del training_label

In [5]:
# Creating a copy of test data so making changes would not alter the initial dataset
#test_df = test_values_data.copy()

In [6]:
# Creating a copy of train data so making changes would not alter the initial dataset
df = train_data.copy()

In [7]:
#test_df.head()

In [8]:
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [9]:
#To see all the columns and rows
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [10]:
#Summary of the test_df to check for missing values - Drope the test data from and split the train dataframe for ML
#test_df.info()

In [11]:
#Summary of the df to check for missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 41 columns):
id                       59400 non-null int64
amount_tsh               59400 non-null float64
date_recorded            59400 non-null object
funder                   55765 non-null object
gps_height               59400 non-null int64
installer                55745 non-null object
longitude                59400 non-null float64
latitude                 59400 non-null float64
wpt_name                 59400 non-null object
num_private              59400 non-null int64
basin                    59400 non-null object
subvillage               59029 non-null object
region                   59400 non-null object
region_code              59400 non-null int64
district_code            59400 non-null int64
lga                      59400 non-null object
ward                     59400 non-null object
population               59400 non-null int64
public_meeting           56066 non-null object
r

In [12]:
# the shape of df
df.shape

(59400, 41)

In [13]:
# Checking for missing values.
df.isnull().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [14]:
# Looking at the values in 'funder' column, one of the columns with missing values
df['funder'].value_counts()
#train_data['funder'].value_counts()>1000

Government Of Tanzania            9084
Danida                            3114
Hesawa                            2202
Rwssp                             1374
World Bank                        1349
Kkkt                              1287
World Vision                      1246
Unicef                            1057
Tasaf                              877
District Council                   843
Dhv                                829
Private Individual                 826
Dwsp                               811
0                                  777
Norad                              765
Germany Republi                    610
Tcrs                               602
Ministry Of Water                  590
Water                              583
Dwe                                484
Netherlands                        470
Hifab                              450
Adb                                448
Lga                                442
Amref                              425
Fini Water               

In [15]:
#t = train_data.copy()
#t = t['funder'].value_counts()
#t = t[t>1000]
#t.index
# = t[t['funder'] == 'Danida']
#en(t)

In [16]:
top_funder = df['funder'].value_counts().head(5).index
top_funder = list(top_funder) 
top_funder

['Government Of Tanzania', 'Danida', 'Hesawa', 'Rwssp', 'World Bank']

In [17]:
# Using a for loop to keep the top 5 values in funder and set the rest to Other
for i, row in df.iterrows():
    if row['funder'] not in top_funder:
        df.loc[i, 'funder'] = 'Other'

In [19]:
# Creating a function to keep top 5 values in funder and set the rest to other.

"""def top_funder(row):
    if row['funder']=='Government Of Tanzania':
        return 'Government Of Tanzania'
    elif row['funder']=='Danida':
        return 'Danida'
    elif row['funder']=='Hesawa':
        return 'Hesawa'
    elif row['funder']=='Rwssp':
        return 'Rwssp'
    elif row['funder']=='World Bank':
        return 'World Bank'    
    else:
        return 'Other'

    
df['funder'] = df.apply(lambda row: top_funder(row), axis=1)"""

"def top_funder(row):\n    if row['funder']=='Government Of Tanzania':\n        return 'Government Of Tanzania'\n    elif row['funder']=='Danida':\n        return 'Danida'\n    elif row['funder']=='Hesawa':\n        return 'Hesawa'\n    elif row['funder']=='Rwssp':\n        return 'Rwssp'\n    elif row['funder']=='World Bank':\n        return 'World Bank'    \n    else:\n        return 'Other'\n\n    \ndf['funder'] = df.apply(lambda row: top_funder(row), axis=1)"

In [20]:
df['funder']

0                         Other
1                         Other
2                         Other
3                         Other
4                         Other
5                         Other
6                         Other
7                         Rwssp
8                         Other
9                         Other
10                        Other
11                       Danida
12                        Other
13                        Other
14                        Other
15                        Other
16                        Other
17                        Other
18                       Hesawa
19                       Danida
20                        Other
21                        Other
22                        Other
23                        Other
24                        Other
25       Government Of Tanzania
26                        Other
27                       Hesawa
28       Government Of Tanzania
29                        Other
30                        Other
31      

In [21]:
#df['funder'] = df['funder'].map({'Government Of Tanzania':'gov', 'Danida':'danida', 
 #                                            'Hesawa':'hesawa', 'Rwssp':'rwssp', })

In [22]:
# Checking to make sure funder values are changed
df['funder'].unique()

array(['Other', 'Rwssp', 'Danida', 'Hesawa', 'Government Of Tanzania',
       'World Bank'], dtype=object)

In [23]:
# Adding a column named 'status_group_vals' to allow the use of a pivot table to check differences between the different funders

"""vals_to_replace = {'functional':2, 'functional needs repair':1,
                   'non functional':0}

df['status_group_vals']  = df.status_group.replace(vals_to_replace)"""

In [24]:
df.columns

Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'quantity_group',
       'source', 'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group', 'status_group', 'status_group_vals'],
      dtype='object')

In [25]:
# Creating a Pivot Table using df and also funder and status_group as index.
piv_table = pd.pivot_table(df,index=['funder','status_group'],
                           values='status_group_vals', aggfunc='count').status_group_vals

piv_table

funder                  status_group           
Danida                  functional                  1713
                        functional needs repair      159
                        non functional              1242
Government Of Tanzania  functional                  3720
                        functional needs repair      701
                        non functional              4663
Hesawa                  functional                   936
                        functional needs repair      232
                        non functional              1034
Other                   functional                 24540
                        functional needs repair     3019
                        non functional             14718
Rwssp                   functional                   805
                        functional needs repair      109
                        non functional               460
World Bank              functional                   545
                        functional needs

In [26]:
funder_status = df.groupby(['funder', 'status_group']).size()

In [27]:
funder_status

funder                  status_group           
Danida                  functional                  1713
                        functional needs repair      159
                        non functional              1242
Government Of Tanzania  functional                  3720
                        functional needs repair      701
                        non functional              4663
Hesawa                  functional                   936
                        functional needs repair      232
                        non functional              1034
Other                   functional                 24540
                        functional needs repair     3019
                        non functional             14718
Rwssp                   functional                   805
                        functional needs repair      109
                        non functional               460
World Bank              functional                   545
                        functional needs

In [28]:
total_danida = funder_status[0] + funder_status[1] + funder_status[2]
percent_functional_danida = (funder_status[0] / total_danida) * 100

In [29]:
print('Percent functional danida: ', round(percent_functional_danida,2))

Percent functional danida:  55.01


In [30]:
# Using Pivot Table, we are going to compare all the funders percentage of functional pumps

total_danida = piv_table[0] + piv_table[1] + piv_table[2]
percent_functional_danida = (piv_table[0] / total_danida) * 100

total_gov = piv_table[3] + piv_table[4] + piv_table[5]
percent_functional_gov = (piv_table[3] / total_gov) * 100

total_hesawa = piv_table[6] + piv_table[7] + piv_table[8]
percent_functional_hesawa = (piv_table[6] / total_hesawa) * 100

total_other = piv_table[9] + piv_table[10] + piv_table[11]
percent_functional_non_gov = (piv_table[9] / total_other) * 100

total_rwssp = piv_table[12] + piv_table[13] + piv_table[14]
percent_functional_rwssp = (piv_table[12] / total_rwssp) * 100

total_world_bank = piv_table[15] + piv_table[16] + piv_table[17]
percent_functional_world_bank = (piv_table[15] / total_world_bank) * 100

print('Percent functional danida: ', round(percent_functional_danida,3))
print('Percent functional gov: ', round(percent_functional_gov,3))
print('Percent functional hesawa: ', round(percent_functional_hesawa,3))
print('Percent functional non gov: ', round(percent_functional_non_gov,3))
print('Percent functional rwssp: ', round(percent_functional_rwssp,3))
print('Percent functional world bank: ', round(percent_functional_world_bank,3))

Percent functional danida:  55.01
Percent functional gov:  40.951
Percent functional hesawa:  42.507
Percent functional non gov:  58.046
Percent functional rwssp:  58.588
Percent functional world bank:  40.4


### There are some clear differences here for functional pumps that will hopefully improve the model

In [31]:
# The next column to inspect is 'installer'.

df.installer.value_counts()

DWE                               17402
Government                         1825
RWE                                1206
Commu                              1060
DANIDA                             1050
KKKT                                898
Hesawa                              840
0                                   777
TCRS                                707
Central government                  622
CES                                 610
Community                           553
DANID                               552
District Council                    551
HESAWA                              539
World vision                        408
LGA                                 408
WEDECO                              397
TASAF                               396
District council                    392
Gover                               383
AMREF                               329
TWESA                               316
WU                                  301
Dmdd                                287


In [32]:
# Creating a function to keep top 5 values in installer and set the rest to other.

def installer_top(row):
    if row['installer']=='DWE':
        return 'dwe'
    elif row['installer']=='Government':
        return 'gov'
    elif row['installer']=='RWE':
        return 'rwe'
    elif row['installer']=='Commu':
        return 'commu'
    elif row['installer']=='DANIDA':
        return 'danida'
    else:
        return 'other'  

df['installer'] = df.apply(lambda row: installer_top(row), axis=1)

In [33]:
# Creating a Pivot Table using df and also installer and status_group as index.
piv_table = pd.pivot_table(df,index=['installer','status_group'],
                           values='status_group_vals', aggfunc='count').status_group_vals

piv_table

installer  status_group           
commu      functional                   724
           functional needs repair       32
           non functional               304
danida     functional                   542
           functional needs repair       83
           non functional               425
dwe        functional                  9433
           functional needs repair     1622
           non functional              6347
gov        functional                   535
           functional needs repair      256
           non functional              1034
other      functional                 20721
           functional needs repair     2187
           non functional             13949
rwe        functional                   304
           functional needs repair      137
           non functional               765
Name: status_group_vals, dtype: int64

In [34]:
# Using Pivot Table, we are going to compare all the installer percentage of functional pumps

total_dwe = piv_table[0] + piv_table[1] + piv_table[2]
percent_functional_dwe = (piv_table[0] / total_dwe) * 100

total_gov = piv_table[3] + piv_table[4] + piv_table[5]
percent_functional_gov = (piv_table[3] / total_gov) * 100

total_hesawa = piv_table[6] + piv_table[7] + piv_table[8]
percent_functional_hesawa = (piv_table[6] / total_hesawa) * 100

total_other = piv_table[9] + piv_table[10] + piv_table[11]
percent_functional_non_gov = (piv_table[9] / total_other) * 100

total_rwssp = piv_table[12] + piv_table[13] + piv_table[14]
percent_functional_rwssp = (piv_table[12] / total_rwssp) * 100

total_world_bank = piv_table[15] + piv_table[16] + piv_table[17]
percent_functional_world_bank = (piv_table[15] / total_world_bank) * 100

print('Percent functional dwe: ', round(percent_functional_dwe,3))
print('Percent functional gov: ', round(percent_functional_gov,3))
print('Percent functional hesawa: ', round(percent_functional_hesawa,3))
print('Percent functional non gov: ', round(percent_functional_non_gov,3))
print('Percent functional rwssp: ', round(percent_functional_rwssp,3))
print('Percent functional world bank: ', round(percent_functional_world_bank,3))

Percent functional dwe:  68.302
Percent functional gov:  51.619
Percent functional hesawa:  54.206
Percent functional non gov:  29.315
Percent functional rwssp:  56.22
Percent functional world bank:  25.207


### As was the case with 'funder', there are some clear differences here that will hopefully improve the model

In [35]:
# The next column to inspect is 'subvillage'.

df.subvillage.value_counts()

Madukani                          508
Shuleni                           506
Majengo                           502
Kati                              373
Mtakuja                           262
Sokoni                            232
M                                 187
Muungano                          172
Mbuyuni                           164
Mlimani                           152
Songambele                        147
Miembeni                          134
Msikitini                         134
1                                 132
Kibaoni                           114
Kanisani                          111
Mapinduzi                         109
I                                 109
Mjimwema                          108
Mjini                             108
Mkwajuni                          104
Mwenge                            102
Mabatini                           98
Azimio                             98
Mbugani                            95
Mission                            95
Bwawani     

In [36]:
# Number of villages 
len(df.subvillage.value_counts())

19287

#### 19287 unique values! The largest values in subvillages don't dominate as was the case with installer and funder. It's probably not worth creating dummy variables for the top 5 values

In [37]:
# Droping subvillage column

df = df.drop('subvillage', axis=1)


In [38]:
# The next column to inspect is 'public_meeting'.

df.public_meeting.value_counts()

True     51011
False     5055
Name: public_meeting, dtype: int64

#### We have Boolean values in this public_meeting column. 

In [39]:
# We need to replace the unknown data in public_meeting column with a string value.

df.public_meeting = df.public_meeting.fillna('Unknown')

In [40]:
# The next column to inspect is 'scheme_management'

df.scheme_management.value_counts()

VWC                 36793
WUG                  5206
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
None                    1
Name: scheme_management, dtype: int64

In [41]:
# Creating a function to keep top 5 values in scheme_management and set the rest to 'other'.

def scheme_top(row):
    if row['scheme_management']=='VWC':
        return 'vwc'
    elif row['scheme_management']=='WUG':
        return 'wug'
    elif row['scheme_management']=='Water authority':
        return 'wtr_auth'
    elif row['scheme_management']=='WUA':
        return 'wua'
    elif row['scheme_management']=='Water Board':
        return 'wtr_brd'
    else:
        return 'other'

df['scheme_management'] = df.apply(lambda row: scheme_top(row), axis=1)

In [42]:
# Creating a Pivot Table using df and also scheme_management and status_group as index.

piv_table = pd.pivot_table(df, index=['scheme_management', 'status_group'],
                           values='status_group_vals', aggfunc='count').status_group_vals

piv_table

scheme_management  status_group           
other              functional                  4627
                   functional needs repair      513
                   non functional              3477
vwc                functional                 18960
                   functional needs repair     2334
                   non functional             15499
wtr_auth           functional                  1618
                   functional needs repair      448
                   non functional              1087
wtr_brd            functional                  2053
                   functional needs repair      111
                   non functional               584
wua                functional                  1995
                   functional needs repair      239
                   non functional               649
wug                functional                  3006
                   functional needs repair      672
                   non functional              1528
Name: status_group_va

In [43]:
# Using Pivot Table, we are going to compare all the scheme_management percentage of functional pumps

total_other = piv_table[0] + piv_table[1] + piv_table[2]
percent_functional_other = (piv_table[0] / total_other) * 100

total_vwc = piv_table[3] + piv_table[4] + piv_table[5]
percent_functional_vwc = (piv_table[3] / total_vwc) * 100

total_wtr_auth = piv_table[6] + piv_table[7] + piv_table[8]
percent_functional_wtr_auth = (piv_table[6] / total_wtr_auth) * 100

total_wtr_brd = piv_table[9] + piv_table[10] + piv_table[11]
percent_functional_wtr_brd = (piv_table[9] / total_wtr_brd) * 100

total_wua = piv_table[12] + piv_table[13] + piv_table[14]
percent_functional_wua = (piv_table[12] / total_wua) * 100

total_wug = piv_table[15] + piv_table[16] + piv_table[17]
percent_functional_wug = (piv_table[15] / total_wug) * 100

print('Percent functional other: ', round(percent_functional_other,3))
print('Percent functional vwc: ', round(percent_functional_vwc,3))
print('Percent functional water authority: ', round(percent_functional_wtr_auth,3))
print('Percent functional water board: ', round(percent_functional_wtr_brd,3))
print('Percent functional wua: ', round(percent_functional_wua,3))
print('Percent functional wug: ', round(percent_functional_wug,3))

Percent functional other:  53.696
Percent functional vwc:  51.532
Percent functional water authority:  51.316
Percent functional water board:  74.709
Percent functional wua:  69.199
Percent functional wug:  57.741


### Water Board Scheme Management have a very high percentage of functional pumps

In [44]:
# The next column to inspect is 'scheme_name'

df.scheme_name.value_counts()

K                                                 682
None                                              644
Borehole                                          546
Chalinze wate                                     405
M                                                 400
DANIDA                                            379
Government                                        320
Ngana water supplied scheme                       270
wanging'ombe water supply s                       261
wanging'ombe supply scheme                        234
I                                                 229
Bagamoyo wate                                     229
Uroki-Bomang'ombe water sup                       209
N                                                 204
Kirua kahe gravity water supply trust             193
Machumba estate pipe line                         185
Makwale water supplied sche                       166
Kijiji                                            161
S                           

In [45]:
len(df.scheme_name.unique())

2697

#### Lots of factors (unique values) and the top 5 or so only represent a fraction of the total values.

In [46]:
# Droping the 'scheme_name' column

df = df.drop('scheme_name', axis=1)

In [47]:
# The final column containing nulls is 'permit'.

df.permit.value_counts()

True     38852
False    17492
Name: permit, dtype: int64

#### We have Boolean values in this public_meeting column.

In [48]:
# We need to replace the unknown data in the 'permit' column with a string value.

df.permit = df.permit.fillna('Unknown')

In [49]:
# Check for missing values in df after going through all of the columns with missing values
df.isnull().sum()

id                       0
amount_tsh               0
date_recorded            0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
num_private              0
basin                    0
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
public_meeting           0
recorded_by              0
scheme_management        0
permit                   0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
source_class             0
waterpoint_type          0
w

### Now there are no nulls in the data set. We can move on to look at columns with string values and modify or remove them as we see fit.

In [50]:
# Checking for columns with string values dtype('object')

str_cols = df.select_dtypes(include = ['object'])
str_cols.apply(lambda x: len(x.unique()))

date_recorded              356
funder                       6
installer                    6
wpt_name                 37400
basin                        9
region                      21
lga                        125
ward                      2092
public_meeting               3
recorded_by                  1
scheme_management            6
permit                       3
extraction_type             18
extraction_type_group       13
extraction_type_class        7
management                  12
management_group             5
payment                      7
payment_type                 7
water_quality                8
quality_group                6
quantity                     5
quantity_group               5
source                      10
source_type                  7
source_class                 3
waterpoint_type              7
waterpoint_type_group        6
status_group                 3
dtype: int64

In [51]:
# Inspecting the first column 'Date recorded' with string values

df.date_recorded.describe()

count          59400
unique           356
top       2011-03-15
freq             572
Name: date_recorded, dtype: object

##### Let's first convert the column to type datetime. Then change the column to represent the number of days since the most recently recorded date we have. The idea being that more recently recorded pumps might be more likely to be functional than non-functional. 

In [52]:
# Converting string values to datetime 

df.date_recorded = pd.to_datetime(df.date_recorded)
df.date_recorded.describe()

count                   59400
unique                    356
top       2011-03-15 00:00:00
freq                      572
first     2002-10-14 00:00:00
last      2013-12-03 00:00:00
Name: date_recorded, dtype: object

##### The most recent data is 2013-12-03. 

In [53]:
# Subtract each date from this point to obtain a 'days_since_recorded' column.

df.date_recorded = pd.datetime(2013, 12, 3) - pd.to_datetime(df.date_recorded)
df.columns = ['days_since_recorded' if x=='date_recorded' else x for x in df.columns]
df.days_since_recorded = df.days_since_recorded.astype('timedelta64[D]').astype(int)
df.days_since_recorded.describe()

count    59400.000000
mean       613.616970
std        334.216374
min          0.000000
25%        297.000000
50%        419.000000
75%        977.000000
max       4068.000000
Name: days_since_recorded, dtype: float64